In [ ]:
import pandas as pd
import re
import numpy as np


In [ ]:
df = pd.read_csv(r'for_table.csv', nrows = 20000)
df = df.dropna(subset = 'response')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ticket_number  20000 non-null  object
 1   response       20000 non-null  object
 2   nps            20000 non-null  object
 3   csat           19133 non-null  object
 4   Term           17855 non-null  object
 5   Tier_2         17855 non-null  object
 6   Tier_1         17855 non-null  object
dtypes: object(7)
memory usage: 1.1+ MB


In [ ]:
#UK phone number
phone_pattern = '((\+44(\s\(0\)\s|\s0\s|\s)?)|0)7\d{3}(\s)?\d{6}'
df['response'] = df['response'].apply(lambda x: re.sub(phone_pattern, '[number_removed]',x ))

#email address
email_pattern = r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+'
df['response'] = df['response'].apply(lambda x: re.sub(email_pattern, '[email_removed]',x ))



In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [ ]:
import datasets
from datasets import Dataset, DatasetDict

prob_df = df['response'].to_frame()

ds = Dataset.from_pandas(prob_df)


from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["response"], truncation = True, padding = True, max_length = 400)


tokenized_datasets = ds.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["response"])
tokenized_datasets.set_format("torch")

In [ ]:
import torch
from torch.utils.data import DataLoader

predict_dataloader = DataLoader(
    tokenized_datasets, shuffle=True, batch_size=8, collate_fn=data_collator
)

In [ ]:
for batch in predict_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([8, 400]),
 'token_type_ids': torch.Size([8, 400]),
 'attention_mask': torch.Size([8, 400])}

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(checkpoint)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

None torch.Size([8, 400, 9])


In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
all_logits = []
input_ids = []
for batch in predict_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    input_ids.append(batch['input_ids'])
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    all_logits.append(logits)

all_logits = torch.cat(all_logits, dim=0)
probs = torch.softmax(all_logits, dim=1).cpu().numpy()
#probs

RuntimeError: ignored

In [ ]:
import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

pipe = nlp = pipeline("ner", model=model, tokenizer=tokenizer)
#dataset = datasets.load_dataset("superb", name="asr", split="test")


# KeyDataset (only *pt*) will simply return the item in the dict returned by the dataset item
# as we're not interested in the *target* part of the dataset. For sentence pair use KeyPairDataset
for out in tqdm(pipe(KeyDataset(ds, "text"))):
    print(out)
    # {"text": "NUMBER TEN FRESH NELLY IS WAITING ON YOU GOOD NIGHT HUSBAND"}
    # {"text": ....}
    # ....

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/5000 [00:00<?, ?it/s]

KeyError: ignored

In [ ]:
KeyDataset(ds)

TypeError: ignored

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [ ]:
# Datasets 2.11.0
from datasets import load_dataset
#dataset = load_dataset("argilla/gutenberg_spacy-ner", split="train")
#Transformers 4.27.4, Torch 2.0.0+cu118,
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    pipeline
)
from transformers.pipelines.pt_utils import KeyDataset

model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

pipe = pipeline(task="ner", model=model, tokenizer=tokenizer, device = device)
results = pipe(KeyDataset(ds, "response"))

original = []
temp_ent = []
entities = []

for idx, extracted_entities in enumerate(results):
    temp_ent = []
    # print('idx: ', idx)
    # print("Original text:\n{}".format(ds[idx]["response"]))
    # print("Extracted entities:")
    #original.append(dataset[idx]["text"])
    for entity in extracted_entities:
        temp_ent.append(entity)
        # print(entity)
    entities.append(temp_ent)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
df['entities'] = entities
df2 = df.explode('entities')

In [ ]:
#df2[['entity_type','entity_start','entity_end']] = np.nan

df1 = df2[df2['entities'].isna() == True]
df2 = df2[df2['entities'].isna() == False]

#entity
df2['entity_type'] = df2['entities'].apply(lambda x:  x['entity'])
#start
df2['entity_start'] = df2['entities'].apply(lambda x:  x['start'])
#end
df2['entity_end'] = df2['entities'].apply(lambda x:  x['end'])


<ipython-input-18-9e70a80c24d2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['entity_type'] = df2['entities'].apply(lambda x:  x['entity'])
<ipython-input-18-9e70a80c24d2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['entity_start'] = df2['entities'].apply(lambda x:  x['start'])
<ipython-input-18-9e70a80c24d2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3644 entries, 0 to 19989
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ticket_number  3644 non-null   object
 1   response       3644 non-null   object
 2   nps            3644 non-null   object
 3   csat           3505 non-null   object
 4   Term           3359 non-null   object
 5   Tier_2         3359 non-null   object
 6   Tier_1         3359 non-null   object
 7   entities       3644 non-null   object
 8   entity_type    3644 non-null   object
 9   entity_start   3644 non-null   int64 
 10  entity_end     3644 non-null   int64 
 11  num            3644 non-null   int64 
dtypes: int64(3), object(9)
memory usage: 370.1+ KB


In [ ]:
df2['num'] = np.where(df2['entity_type'] == 'B-PER', 1, 2)

df2 = df2.sort_values('num', ascending = True).drop_duplicates(['ticket_number','Term'])

<ipython-input-19-6f6780d8b3e0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['num'] = np.where(df2['entity_type'] == 'B-PER', 1, 2)


In [ ]:
df2.to_csv(r'entity_append.csv')

In [ ]:
df2 = df2[df2['entity_type'] == 'B-PER']

In [ ]:
def remove_names(x,start,end):
  x = x[:start] + '[Removed Named] ' + x[end:]
  return x

df2['check'] = df2.apply(lambda x: remove_names(x['response'], x['entity_start'], x['entity_end']), axis = 1)
df2.to_csv('check.csv')

In [ ]:
check = df[df['entity_type'] == 'B-PER']
check.to_csv('check.csv')

In [ ]:
dataset

Dataset({
    features: ['text', 'tokens', 'prediction', 'prediction_agent', 'annotation', 'annotation_agent', 'id', 'metadata', 'status', 'event_timestamp', 'metrics', 'vectors'],
    num_rows: 100
})

In [ ]:
ds

Dataset({
    features: ['response'],
    num_rows: 5000
})